In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

X = torch.rand(2, 20)
net(X)

tensor([[-0.0100, -0.1575, -0.2103, -0.0298, -0.0135,  0.1147,  0.1274,  0.0367,
          0.1032,  0.0039],
        [ 0.0922, -0.0311, -0.2422,  0.0734,  0.0860,  0.1738,  0.2845,  0.1084,
          0.0335, -0.0271]], grad_fn=<AddmmBackward0>)

In [2]:
# 自定义块
class MLP(nn.Module):
    # 用模型参数声明层
    def __init__(self):
        # 执行必要的初始化
        super().__init__() # 调用父类初始化（__init__）函数
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层

    # 定义模型的前向传播：如何根据输入X返回所需的模型输出 
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)

tensor([[-0.1474, -0.0197, -0.0215, -0.1163,  0.2643, -0.0522,  0.0095, -0.1013,
         -0.0057,  0.0906],
        [-0.0435, -0.0493,  0.0131, -0.1826,  0.3554,  0.0518,  0.0038, -0.2939,
          0.1237,  0.0181]], grad_fn=<AddmmBackward0>)

In [3]:
# 自定义顺序块
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # module为Module子类的一个实例，
            # 将它保存在‘Module’类的成员变量_modules中，_modules类型为OrderedDict
            self._modules[str(idx)] = module
    
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0784,  0.1624, -0.0965,  0.1002,  0.0664,  0.0212,  0.0761,  0.0186,
          0.1154,  0.0304],
        [ 0.1132,  0.1143,  0.1363,  0.1142,  0.0152,  0.0719,  0.0892,  0.1192,
          0.1020,  0.1101]], grad_fn=<AddmmBackward0>)

In [4]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层，相当于两个全连接层共享参数
        X = self.linear(X)
        
        # 插入python控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0543, grad_fn=<SumBackward0>)

In [5]:
# 嵌套块
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU(),
        )
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(
    NestMLP(), 
    nn.Linear(16, 20),
    FixedHiddenMLP(),
)

chimera(X)

tensor(0.3540, grad_fn=<SumBackward0>)